## 注意力机制Attention:

### 简介:

这一部分是Transformer模型的核心部分,以下部分逐步给出实现过程中可能用到的一些矩阵运算的原理， 以下代码均不需要大家实现,希望大家阅读代码以及下列文档中的信息:

https://arxiv.org/abs/1706.03762

https://jalammar.github.io/illustrated-transformer/

理解Attention的运行机制以及实现过程的数学技巧，完成最后的主文件中的HeadAttention(),MultiHeadAttention()部分。

我们虚构一组输入数据的Embedding用于这部分讲解：

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
B, T, C = 1, 8, 16   ## B: batch size 一次训练的数据量, T: context length 前文token数, C: embedding length 隐变量长度
inputData = torch.rand(size=(B,T,C))

for i in range(T):
    print(f"Embedding of {i}th position:\n {inputData[0,i]}")


Embedding of 0th position:
 tensor([0.8746, 0.3950, 0.7806, 0.2509, 0.9771, 0.0925, 0.3809, 0.7409, 0.8331,
        0.2836, 0.2466, 0.3414, 0.8576, 0.2416, 0.9422, 0.8316])
Embedding of 1th position:
 tensor([0.6409, 0.0979, 0.1720, 0.4029, 0.4333, 0.1093, 0.4451, 0.4334, 0.9658,
        0.8726, 0.9698, 0.9580, 0.0643, 0.1569, 0.7886, 0.6275])
Embedding of 2th position:
 tensor([0.1910, 0.3933, 0.5726, 0.3686, 0.1319, 0.4809, 0.4712, 0.7410, 0.6443,
        0.9330, 0.5351, 0.9350, 0.4274, 0.1378, 0.5169, 0.9945])
Embedding of 3th position:
 tensor([0.4499, 0.8932, 0.7088, 0.9068, 0.2618, 0.9636, 0.7131, 0.8136, 0.4323,
        0.1111, 0.5683, 0.3407, 0.3544, 0.7613, 0.1456, 0.0036])
Embedding of 4th position:
 tensor([0.5469, 0.3589, 0.1967, 0.3021, 0.3605, 0.0397, 0.5780, 0.4466, 0.0783,
        0.1008, 0.0628, 0.4282, 0.6583, 0.2024, 0.6971, 0.7359])
Embedding of 5th position:
 tensor([0.9586, 0.6752, 0.2102, 0.7496, 0.8694, 0.0318, 0.6047, 0.0450, 0.5046,
        0.7877, 0.7195, 0.0

Attention从直观上可以理解为对前文各个位置信息的融合以获得当前语境所需的信息。 一个最简单的融合方式为对前文Embedding加权求和作为当前位置的信息。

我们计算第i个位置的融合后的embedding:

假设前i个位置的embedding的权重相同，均为1/i，即更新后第i个位置embedding为前文所有位置embedding的平均值：

In [2]:
def Attention_version1(contextEmbeddings):
    for i in range(T):
        context_embeddings = contextEmbeddings[0,:i+1,:] ## shape [i+1, C]
        new_embedding_for_i = torch.mean(context_embeddings,dim=0)
        contextEmbeddings[0,i] = new_embedding_for_i
    return contextEmbeddings

print("Embedding of Data after aggregate context embedding:\n", Attention_version1(inputData))

Embedding of Data after aggregate context embedding:
 tensor([[[0.8746, 0.3950, 0.7806, 0.2509, 0.9771, 0.0925, 0.3809, 0.7409,
          0.8331, 0.2836, 0.2466, 0.3414, 0.8576, 0.2416, 0.9422, 0.8316],
         [0.7577, 0.2465, 0.4763, 0.3269, 0.7052, 0.1009, 0.4130, 0.5871,
          0.8995, 0.5781, 0.6082, 0.6497, 0.4609, 0.1992, 0.8654, 0.7296],
         [0.6078, 0.3449, 0.6098, 0.3154, 0.6047, 0.2247, 0.4217, 0.6897,
          0.7923, 0.5982, 0.4633, 0.6420, 0.5820, 0.1929, 0.7748, 0.8519],
         [0.6725, 0.4699, 0.6439, 0.4500, 0.6372, 0.3454, 0.4822, 0.7078,
          0.7393, 0.3927, 0.4716, 0.4935, 0.5637, 0.3487, 0.6820, 0.6042],
         [0.6919, 0.3630, 0.5415, 0.3291, 0.6569, 0.1607, 0.4552, 0.6344,
          0.6685, 0.3907, 0.3705, 0.5110, 0.6245, 0.2369, 0.7923, 0.7507],
         [0.7605, 0.4158, 0.5437, 0.4036, 0.7418, 0.1593, 0.4596, 0.5675,
          0.7396, 0.5052, 0.4800, 0.4401, 0.5569, 0.2112, 0.7231, 0.7747],
         [0.7555, 0.3797, 0.5239, 0.3246, 0.7154, 0.

我们将上述的mean操作换为等价的矩阵运算，以i=3 为例：

new_embedding_for_3 = torch.mean(contextEmbeddings[0,:3+1],dim=0)

等价于(@ 是矩阵乘法):

new_embedding_for_3 = contextEmbeddings[0] @ torch.tensor([1/4,1/4,1/4,1/4,0,0,0,0])

In [3]:
def Attention_version2(contextEmbeddings):
    for i in range(T):
        weight = torch.cat((torch.ones(i+1) / (i+1),torch.zeros(T-i-1,dtype=torch.float)),dim=0)
        contextEmbeddings[0,i] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version2: ",torch.all(Attention_version1(inputData) == Attention_version2(inputData)).item())

Attention_version1 equivalent to Attention_version2:  True


接下来我们用矩阵运算进一步简化上述运算，移除其中的for循环:

其中 weight = torch.tril(torch.ones(T,T)) 得到:

[[1., 0., 0., 0., 0., 0., 0., 0.],

 [1., 1., 0., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 1.]]
 
表示前文的求和权重相同都为一。

weight = weight.masked_fill(weight==0,float("-inf"))

weight = F.softmax(weight)

这两行用于归一化weight,即每一次加权求和的权重和为1，具体详见Softmax公式,我们可得到：

[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],

[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],

[0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],

[0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]


In [4]:
def Attention_version3(contextEmbeddings):
    B, T, C = contextEmbeddings.shape
    weight = torch.tril(torch.ones(T,T))
    print("weight of context embeddings:\n",weight)
    weight = weight.masked_fill(weight==0,float("-inf"))
    weight = F.softmax(weight,dim=1)
    print("weight of context embeddings after regularization:\n",weight)
    contextEmbeddings[0] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version3: ",torch.all(Attention_version1(inputData) == Attention_version3(inputData)).item())

weight of context embeddings:
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
weight of context embeddings after regularization:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.125

最后，我们确定计算weight的方法，上述三个版本的weight都是假定所有前文信息的重要程度相同,在大语言模型中，我们希望有一个灵活的方式计算前文信息对应当前语境的重要程度，为此Transformer引入了Query，Key，Value:

其中Query可以理解为当前语境对于前文信息的需求，Key可以理解为前文包含信息的索引，Value为前文所包含的信息。

Query 和 Key 用来计算信息融合的weight.

如何计算Query和Key，并用他们计算weight对Value加权求和是这次实验的重点内容，这里不能给出大家具体代码，希望大家参见Attention is All you need原论文以及助教提供的文档最后的参考链接学习这部分。

利于Query和Key得出的是信息相关性，我们需要遮盖住下文的信息(生成第i个token时，只可以使用0到i-1处的信息)，并且要对相关性归一化使之可以作为weight。这里利于Attension_version3()中的结论给出如何对计算出来的相关性加掩码和归一化:


In [5]:
def weight_mask_and_normalization(weight):
    tril = torch.tril(torch.ones_like(weight))
    weight = weight.masked_fill(tril == 0, float("-inf"))
    weight = F.softmax(weight,dim=-1)
    return weight

weight = torch.rand(T,T)
print("weight before mask and normalize:\n",weight)
print("weight after mask and normalize:\n",weight_mask_and_normalization(weight))

weight before mask and normalize:
 tensor([[0.7479, 0.5198, 0.3857, 0.8638, 0.0275, 0.1866, 0.9334, 0.8984],
        [0.9416, 0.5996, 0.9691, 0.0764, 0.0773, 0.9818, 0.4806, 0.9050],
        [0.9041, 0.4469, 0.4609, 0.7194, 0.4701, 0.9888, 0.4267, 0.9623],
        [0.7619, 0.0338, 0.4130, 0.1626, 0.7994, 0.9799, 0.9258, 0.9302],
        [0.7575, 0.0044, 0.8507, 0.2724, 0.1523, 0.3918, 0.7639, 0.9619],
        [0.3386, 0.4334, 0.8066, 0.8829, 0.1460, 0.2816, 0.0251, 0.6885],
        [0.2504, 0.9890, 0.5016, 0.6060, 0.6594, 0.1603, 0.5360, 0.4624],
        [0.2615, 0.5867, 0.3610, 0.2111, 0.8150, 0.7288, 0.6525, 0.2659]])
weight after mask and normalize:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5847, 0.4153, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4396, 0.2783, 0.2822, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3653, 0.1764, 0.2577, 0.2006, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2681, 0.1262, 0.2943, 0.1650, 0